# NUSA Demo of Fama French Factor Model

In [10]:
import pandas as pd
import numpy as np
# use any other libraries you may need

Read the contents of **cleaned_factset_data.csv**  into a Pandas Dataframe called **df** and drop any rows with NaN values. Note that the **CAP** column values are Strings with commas to denote thousands, so convert all the values in the column to Floats.

In [11]:
df = pd.read_csv('cleaned_factset_data.csv')
df.head()

,Ticker,Company Name,monthly_return,capm_beta,book_price,CAP,GPM
0,DDD,3D Systems Corporation,-6.02,1.555648,0.436308,1523.9963,48.936516
1,MMM,3M Company,4.50,1.079156,0.074971,125018.1300,49.739280
2,EGHT,"8x8, Inc.",-1.81,0.366954,0.236263,1241.3590,75.486600
3,AOS,A. O. Smith Corporation,-1.52,1.536893,0.147579,11333.7530,41.665737
4,SHLM,"A. Schulman, Inc.",9.18,1.600787,0.033661,1006.2639,16.560259


To reduce the impact of outliers caused by the few number of large cap companies, add a new column to **df** called **log_mktcap** and populate it with the log of each value in **CAP**. 

In [12]:
df['log_mktcap'] = np.log(df['CAP'])

In [13]:
df

,Ticker,Company Name,monthly_return,capm_beta,book_price,CAP,GPM,log_mktcap
0,DDD,3D Systems Corporation,-6.02,1.555648,0.436308,1523.99630,48.936516,7.329091
1,MMM,3M Company,4.50,1.079156,0.074971,125018.13000,49.739280,11.736214
2,EGHT,"8x8, Inc.",-1.81,0.366954,0.236263,1241.35900,75.486600,7.123962
3,AOS,A. O. Smith Corporation,-1.52,1.536893,0.147579,11333.75300,41.665737,9.335541
4,SHLM,"A. Schulman, Inc.",9.18,1.600787,0.033661,1006.26390,16.560259,6.914000
...,...,...,...,...,...,...,...,...
1501,ZBRA,Zebra Technologies Corporation Class A,3.12,1.604373,0.129115,5766.58000,39.653730,8.659834
1502,ZBH,"Zimmer Biomet Holdings, Inc.",0.98,1.182440,0.396316,23663.88900,60.496624,10.071705
1503,ZION,Zions Bancorporation,NaN,1.470795,0.733912,9422.41200,NaN,9.150846
1504,ZTS,"Zoetis, Inc. Class A",-1.51,1.016201,0.047275,31104.16800,64.566284,10.345097


Then calculate the z-score of each of the numeric columns and put the results into new columns with **'zscore_'** prepended to each original column name. 


The z-score formula is:

|      $Z = \frac{x - \mu}{\sigma}$

Where $\mu$ is the column mean, $\sigma$ is the column standard deviation, and $x$ is the observed value.


In [16]:
def z_score(col):
    df['zscore_' + col] = (df[col] - df[col].mean()) / df[col].std()
    return df

for column in df.columns[2:]:
    z_score(column)

df.head()

,Ticker,Company Name,monthly_return,capm_beta,book_price,CAP,GPM,log_mktcap,zscore_monthly_return,zscore_capm_beta,zscore_book_price,zscore_CAP,zscore_GPM,zscore_log_mktcap,zscore_zscore_monthly_return,zscore_zscore_capm_beta,zscore_zscore_book_price,zscore_zscore_CAP,zscore_zscore_GPM,zscore_zscore_log_mktcap
0,DDD,3D Systems Corporation,-6.02,1.555648,0.436308,1523.9963,48.936516,7.329091,-0.927340,0.618140,-0.046824,-0.302299,0.558255,-0.691310,-0.927340,0.618140,-0.046824,-0.302299,0.558255,-0.691310
1,MMM,3M Company,4.50,1.079156,0.074971,125018.1300,49.739280,11.736214,0.938459,-0.066353,-0.747973,2.053508,0.594841,2.182752,0.938459,-0.066353,-0.747973,2.053508,0.594841,2.182752
2,EGHT,"8x8, Inc.",-1.81,0.366954,0.236263,1241.3590,75.486600,7.123962,-0.180666,-1.089453,-0.434997,-0.307691,1.768273,-0.825083,-0.180666,-1.089453,-0.434997,-0.307691,1.768273,-0.825083
3,AOS,A. O. Smith Corporation,-1.52,1.536893,0.147579,11333.7530,41.665737,9.335541,-0.129232,0.591199,-0.607082,-0.115166,0.226890,0.617177,-0.129232,0.591199,-0.607082,-0.115166,0.226890,0.617177
4,SHLM,"A. Schulman, Inc.",9.18,1.600787,0.033661,1006.2639,16.560259,6.914000,1.768491,0.682984,-0.828131,-0.312176,-0.917290,-0.962008,1.768491,0.682984,-0.828131,-0.312176,-0.917290,-0.962008


Winsorize the data in the **'zscore'** columns at the 1st and 99th percentiles. 
(Censor the outliers, set any values less than the 1st percentile to the value of the 1st percentile and any values greater than the 99th percentile to the value at the 99th percentile).

In [18]:
from scipy.stats import zscore
from scipy.stats.mstats import winsorize

Run a **weighted least squares regression** using the standardized, winsorized data as explanatory variables and the monthly returns as the dependent.

Write a sentence or two interpreting the results of the regression, what do the coefficients mean and are they statistically significant?